In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# === CONFIG ===
model_path = "runs/segment/Lane_V3s2/weights/best.pt"  # Path to your trained model
image_path = "test_images/multi.png"           # Path to test image
output_path = "output/demo_1.jpeg"    # Save path

# === Load model ===
model = YOLO(model_path)  # Load your YOLOv8/v11 segmentation model

# === Load image ===
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Image not found at {image_path}")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# === Inference ===
results = model(img_rgb)[0]  # Inference on single image

# === Alpha blending setup ===
overlay = img.copy()
alpha = 0.5  # Transparency of mask overlays
mask_color = (0, 255, 0)  # Green mask

# === Resize and blend masks ===
if results.masks is not None:
    masks = results.masks.data.cpu().numpy()  # Shape: [N, H_mask, W_mask]
    orig_h, orig_w = img.shape[:2]

    for mask in masks:
        # Resize the mask to the original image size
        mask_resized = cv2.resize((mask * 255).astype(np.uint8), (orig_w, orig_h))

        # Create a color mask
        color_mask = np.zeros_like(img, dtype=np.uint8)
        color_mask[:, :, 1] = mask_resized  # Green channel

        # Blend with original image
        overlay = cv2.addWeighted(overlay, 1, color_mask, alpha, 0)

# === Final result ===
img_result = overlay

# === Save result ===
os.makedirs(Path(output_path).parent, exist_ok=True)
cv2.imwrite(output_path, img_result)
print(f"✓ Result saved to: {output_path}")


/home/atlanta/miniconda3/envs/Custom_train/lib/python3.10/site-packages/torch/serialization.py:1488: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


WARNING ⚠️ The file 'yolopv2.pt' appears to be improperly saved or formatted. For optimal results, use model.save('filename.pt') to correctly save YOLO models.


AttributeError: 'RecursiveScriptModule' object has no attribute 'fuse'

In [2]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# === CONFIG ===
model_path = "runs/segment/Lane_V3s2/weights/best.pt"   # Path to trained YOLO model
video_path = "test_video/test.mp4"                   # Path to test video
output_path = "output/demo_video.mp4"                 # Output video path

# === Load YOLO model ===
model = YOLO(model_path)

# === Open video ===
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Video not found at {video_path}")

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
os.makedirs(Path(output_path).parent, exist_ok=True)
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# === Process video frames ===
alpha = 0.5  # Mask transparency
mask_color = (0, 255, 0)  # Green mask

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Inference
    results = model(frame_rgb)[0]

    overlay = frame.copy()
    if results.masks is not None:
        masks = results.masks.data.cpu().numpy()
        for mask in masks:
            mask_resized = cv2.resize((mask * 255).astype(np.uint8), (width, height))
            color_mask = np.zeros_like(frame, dtype=np.uint8)
            color_mask[:, :, 1] = mask_resized  # Green channel
            overlay = cv2.addWeighted(overlay, 1, color_mask, alpha, 0)

    # Write frame to output video
    out.write(overlay)

# === Cleanup ===
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✓ Video result saved to: {output_path}")



0: 384x640 2 lanes, 5.1ms
Speed: 2.5ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 3.7ms
Speed: 1.3ms preprocess, 3.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 3.7ms
Speed: 1.3ms preprocess, 3.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 4.0ms
Speed: 1.8ms preprocess, 4.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 4.2ms
Speed: 1.2ms preprocess, 4.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 3.6ms
Speed: 1.2ms preprocess, 3.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 4.1ms
Speed: 2.1ms preprocess, 4.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 lanes, 4.1ms
Speed: 2.2ms preprocess, 4.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 la

In [8]:
import torch
import cv2
import numpy as np
from pathlib import Path

# === Device selection ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load TorchScript model ===
model = torch.jit.load("yolopv2.pt", map_location=device)
model.eval()

# === Read image ===
img_path = "test_images/multi.png"
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError(f"Image not found at {img_path}")

# Keep original for overlay
orig_img = img.copy()

# === Resize to model's expected size ===
target_w, target_h = 640, 384  # common YOLOPv2 size
img_resized = cv2.resize(img, (target_w, target_h))
img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

# === Preprocess ===
img_tensor = torch.from_numpy(img_rgb).permute(2, 0, 1).float().unsqueeze(0) / 255.0
img_tensor = img_tensor.to(device)

# === Inference ===
with torch.no_grad():
    outputs = model(img_tensor)

# === Debug output structure ===
print(f"Model returned {len(outputs) if isinstance(outputs, (tuple, list)) else 1} outputs")
if isinstance(outputs, (tuple, list)):
    for i, out in enumerate(outputs):
        if torch.is_tensor(out):
            print(f"Output[{i}]: shape={tuple(out.shape)}, dtype={out.dtype}")
        else:
            print(f"Output[{i}]: {type(out)}")
else:
    print(f"Single output: shape={tuple(outputs.shape)}")

# === Try to extract drivable area and lane masks ===
def process_mask(mask_tensor, orig_shape):
    """Converts model mask tensor to uint8 resized mask."""
    if mask_tensor is None:
        return None
    mask = mask_tensor.squeeze().cpu().numpy()  # remove batch dim
    if mask.ndim == 3:  # (C, H, W)
        mask = mask[0]  # take first channel
    mask_bin = (mask > 0.5).astype(np.uint8) * 255
    return cv2.resize(mask_bin, (orig_shape[1], orig_shape[0]), interpolation=cv2.INTER_NEAREST)

drive_area_mask = None
lane_line_mask = None

if isinstance(outputs, (tuple, list)) and len(outputs) >= 3:
    drive_area_mask = process_mask(outputs[1], orig_img.shape)
    lane_line_mask = process_mask(outputs[2], orig_img.shape)

# === Create overlay ===
overlay = orig_img.copy()
if drive_area_mask is not None:
    overlay[drive_area_mask > 0] = (0, 255, 0)  # green
if lane_line_mask is not None:
    overlay[lane_line_mask > 0] = (0, 0, 255)  # red

# === Blend with original ===
alpha = 0.5
blended = cv2.addWeighted(orig_img, 1 - alpha, overlay, alpha, 0)

# === Save result ===
output_path = Path("yolopv2_result.jpg")
cv2.imwrite(str(output_path), blended)
print(f"✓ Result saved to {output_path.resolve()}")


Model returned 3 outputs
Output[0]: <class 'tuple'>
Output[1]: shape=(1, 2, 384, 640), dtype=torch.float32
Output[2]: shape=(1, 1, 384, 640), dtype=torch.float32
✓ Result saved to /home/atlanta/Downloads/Lane_Training/yolopv2_result.jpg
